In [106]:
from hushh import catalog
from hushh.hcf.VibeMode import VibeMode
import numpy as np
from sentence_transformers import util

In [114]:
def perform_search(query):
    cat = catalog.read_hcf("catalog.hcf")
    pv = cat.ProductVibes()
    all_embeddings = []
    
    embeddings = {
        VibeMode.ProductText : [], 
        VibeMode.ProductImage : []
    }
    
    for idx in range(0,pv.FlatBatchesLength()):
        batch = pv.FlatBatches(idx)
        embs = batch.FlatTensorAsNumpy()
        embs = embs.reshape(batch.ShapeAsNumpy())
        embeddings[batch.VibeMode()].append(embs)
    
    for mode in embeddings.keys():
        embeddings[mode] = np.concatenate(embeddings[mode])
        
    query = "shoes"
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    inputs = tokenizer([query], padding=True, return_tensors="pt")
    query_emb = model.get_text_features(**inputs)
    hits = util.semantic_search(query_emb, embeddings[VibeMode.ProductText], top_k=3)[0]
    return cat.ProductVibes().Products(hits[0]['corpus_id']).Description()

In [117]:
perform_search("shoes")

b'Enroute Men Black Shoes'

In [116]:
%timeit -n 3 perform_search("shoes")

398 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
